In [1]:
from  langchain_community.document_loaders import PyMuPDFLoader

In [2]:
loader = PyMuPDFLoader("main.pdf")

In [3]:
documents = loader.load()

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [6]:
documents = text_splitter.split_documents(documents)

In [9]:
import os
os.environ['OPENAI_API_KEY'] = ''
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

In [10]:
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [11]:
from langchain.tools.retriever import create_retriever_tool

In [12]:
backtraderSearch = create_retriever_tool(
    retriever,
    "backtrader_search",
    "This tool contains the Documentation for the backtrader library which we are going to be using, when you are given a strategy, use this tool to see what indicators you are going to use, and how will you make a class using those indicators.",
)

In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI, OpenAI
tools = [backtraderSearch]

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You will always be given a trading strategy as an input, the data will be provided to you, so you only hae to take help of the backtrader_search tool to see how will you make the class which I will later copy and paste and run it using backtrader api. Keep sure you are only making the class and nothing else. Also make the class such that when I run the cerebro engine I see what is hapening every day, that is after you are done with capturing the logic, make logs of the trade every day, example: class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()
                
                see here in this very basic stategy we can see that how the buy/sell is printed, you have to do this for every class you make. Also make sure your position is always there, that is if you are at a buy position you can only sell, if you are at a sell position you can only buy next. Also make sure that array don't go out of range and also the function data.close is used properly.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [17]:
llm_with_tools = model.bind_tools(tools)

In [18]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
# input
# input thing



In [62]:
ans = list(agent_executor.stream({"input": """My stratgey is that whenever the sma50 crosses sma200 we buy else we sell"""}))



> Entering new AgentExecutor chain...

Invoking: `backtrader_search` with `{'query': 'sma'}`


16/08/24, 11:03 PM
Indicators - ta-lib - Backtrader
Page 5 of 19
https://www.backtrader.com/docu/talib/talib/
SMA
Output
EMA
$ ./talibtest.py --plot --ind sma

16/08/24, 11:03 PM
Indicators - Reference - Backtrader
Page 143 of 187
https://www.backtrader.com/docu/indautoref/
PlotLines:
smma:
top:
_samecolor (True)
bot:
_samecolor (True)
SmoothedMovingAverageOscillator
Alias:
SmoothedMovingAverageOsc, SMMAOscillator, SMMAOsc, WilderMAOscillator, WilderMAOsc,
MovingAverageSmoothedOscillator, MovingAverageSmoothedOsc, MovingAverageWilderOscillator,
MovingAverageWilderOsc, ModiSedMovingAverageOscillator, ModiSedMovingAverageOsc
Oscillation of a SmoothedMovingAverage around its data
Lines:
smma
Params:
period (30)
PlotInfo:
plot (True)
plotmaster (None)
legendloc (None)
subplot (True)
plotname ()
plotskip (False)
plotabove (False)
plotlinelabels (False)
plotlinevalues (True)
plotvaluetags (True)


In [63]:
print(ans[-1]['output'])

Here's the class for your strategy, which involves buying when the 50-period SMA crosses above the 200-period SMA and selling when it crosses below:

```python
import backtrader as bt

class SMA50_200CrossStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Add indicators
        self.sma50 = bt.indicators.SimpleMovingAverage(self.datas[0], period=50)
        self.sma200 = bt.indicators.SimpleMovingAverage(self.datas[0], period=200)
        self.crossover = bt.indicators.CrossOver(self.sma50, self.sma200)

    def next(self):
        # Log the closing price
        self.log('Close, %.2f' % self.dataclose[0])
        
        if self.position:  # Check if we have an open position
            if self.crossover < 0:  # If the 50 SMA crosses below the 200 SMA
                self.log('SELL CREATE, %.2f' % s

In [26]:
print(ans)

My purpose is to assist you in generating Backtrader strategy classes based on the given trading strategies. Once you provide a specific trading strategy, I will help translate that into a Python class compatible with the Backtrader library. This class will include logging functionality to track trades and daily activities, and ensure that positions are correctly managed according to the provided strategy.


In [27]:
import yfinance as yf
import pandas as pd

symbol = "RELIANCE.NS"
interval = "1d"
start_date = "2020-08-11"
end_date = "2024-08-17"

df = yf.download(tickers=symbol, interval=interval, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [28]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-11,1970.419678,1993.679321,1938.622314,1969.496704,1942.791626,23414153
2020-08-12,1957.682251,1979.834351,1944.575684,1963.774048,1937.146729,15132308
2020-08-13,1964.143311,1974.203979,1947.529297,1958.651489,1932.093506,12415161
2020-08-14,1959.112915,1990.910278,1928.377075,1951.036743,1924.582031,16612317
2020-08-17,1942.914307,1942.914307,1910.701660,1930.315430,1904.141724,15489533
...,...,...,...,...,...,...
2024-08-09,2920.000000,2953.000000,2912.000000,2948.600098,2938.626465,3124888
2024-08-12,2940.000000,2946.000000,2915.500000,2921.250000,2911.368896,4092292
2024-08-13,2921.500000,2940.149902,2916.000000,2927.250000,2917.348633,3132532


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 994 entries, 2020-08-11 to 2024-08-16
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       994 non-null    float64
 1   High       994 non-null    float64
 2   Low        994 non-null    float64
 3   Close      994 non-null    float64
 4   Adj Close  994 non-null    float64
 5   Volume     994 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 54.4 KB


In [30]:
import backtrader as bt
cerebro = bt.Cerebro()


In [31]:
data_feed = bt.feeds.PandasData(
    dataname=df)

cerebro.adddata(data_feed)

In [66]:
import backtrader as bt

class SMA50_200CrossStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Add indicators
        self.sma50 = bt.indicators.SimpleMovingAverage(self.datas[0], period=50)
        self.sma200 = bt.indicators.SimpleMovingAverage(self.datas[0], period=200)
        self.crossover = bt.indicators.CrossOver(self.sma50, self.sma200)

    def next(self):
        # Log the closing price
        self.log('Close, %.2f' % self.dataclose[0])
        
        if self.position:  # Check if we have an open position
            if self.crossover < 0:  # If the 50 SMA crosses below the 200 SMA
                self.log('SELL CREATE, %.2f' % self.datas[0].close[0])
                self.sell()
        else:
            if self.crossover > 0:  # If the 50 SMA crosses above the 200 SMA
                self.log('BUY CREATE, %.2f' % self.datas[0].close[0])
                self.buy()


In [67]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SMA50_200CrossStrategy)
cerebro.adddata(data_feed)
cerebro.broker.setcash(30000)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 30000.00


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_StrategyBase_Strategy_SMA50_200CrossStrategy' object has no attribute 'dataclose'

In [61]:
fig = cerebro.plot(style='bar', volume=False)[0][0]
fig.set_size_inches(12, 8)
fig.savefig('backtrader_plot.png', dpi=300)


<IPython.core.display.Javascript object>